<a href="https://colab.research.google.com/github/sridevibonthu/EVA/blob/master/s4/assignment_version3_15_4k_99_47.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #block1
        self.conv1 = nn.Conv2d(1, 10, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(10)

        self.conv2 = nn.Conv2d(10, 16, 3, bias=False)
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 32, 3, bias=False)
        self.bn3 = nn.BatchNorm2d(32)

        self.pool1 = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(32, 10, 1, bias=True)
        self.bn4 = nn.BatchNorm2d(10)

        self.conv5 = nn.Conv2d(10, 16, 3, bias=False)
        self.bn5 = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 16, 3, bias=False)
        self.bn6 = nn.BatchNorm2d(16)

        self.conv7 = nn.Conv2d(16, 32, 3, bias=False)
        self.bn7 = nn.BatchNorm2d(32)
        self.do = nn.Dropout2d(0.05)

        self.conv8 = nn.Conv2d(32, 10, 1, bias=False)

        self.gap = nn.AvgPool2d(5)

    def forward(self, x):
        #Block1 -Moves from 28X28X1 to 11X11X10
        x = self.pool1(self.do(self.bn3(F.relu(self.conv3(self.do(self.bn2(F.relu(self.conv2(self.do(self.bn1(F.relu(self.conv1(x)))))))))))))
        x = self.do(self.bn4(F.relu(self.conv4(x))))
        #Block 2 - 11X11X10 to 5X5X10  then GAP
        x = self.do(self.bn7(F.relu(self.conv7(self.do(self.bn6(F.relu(self.conv6(self.do(self.bn5(F.relu(self.conv5(x))))))))))))
        x = self.gap(self.conv8(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
       BatchNorm2d-2           [-1, 10, 26, 26]              20
         Dropout2d-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,440
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           4,608
       BatchNorm2d-8           [-1, 32, 22, 22]              64
         Dropout2d-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]             330
      BatchNorm2d-12           [-1, 10, 11, 11]              20
        Dropout2d-13           [-1, 10, 11, 11]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEPOCH : [{}], Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.02f}%)\n'.format(
        epoch,test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)

  0%|          | 0/469 [00:00<?, ?it/s]


EPOCH : [1], Test set: Average loss: 0.0573, Accuracy: 9851/10000 (98.51%)



  0%|          | 0/469 [00:00<?, ?it/s]


EPOCH : [2], Test set: Average loss: 0.0376, Accuracy: 9891/10000 (98.91%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


EPOCH : [17], Test set: Average loss: 0.0189, Accuracy: 9935/10000 (99.35%)



  0%|          | 0/469 [00:00<?, ?it/s]


EPOCH : [18], Test set: Average loss: 0.0183, Accuracy: 9939/10000 (99.39%)



loss=0.1332649439573288 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.92it/s]



EPOCH : [19], Test set: Average loss: 0.0178, Accuracy: 9947/10000 (99.47%)

